In [168]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [169]:


train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data["Cabin"] = train_data["Cabin"].fillna("Empty")
train_data["Age"] = train_data["Age"].fillna(train_data["Age"].mean())
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Empty,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Empty,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Empty,S


In [170]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data["Cabin"] = test_data["Cabin"].fillna("Empty")
test_data["Age"] = test_data["Age"].fillna(test_data["Age"].mean())
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Empty,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,Empty,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Empty,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,Empty,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,Empty,S


In [171]:
women = train_data.loc[train_data.Sex == "female"]["Survived"]
rate_women = sum(women)/len(women)



print("% of women who survived:", rate_women)



% of women who survived: 0.7420382165605095


In [172]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("amount of men:", len(train_data[train_data["Sex"] == "male"]))

#577 males ~109 survived #314 females ~233 survived
#pclass - groshi, sibsp 	-  of siblings / spouses aboard the Titanic,
#parch 	# of parents / children aboard the Titanic
#cabin 	Cabin numbe
survived_men = train_data[(train_data["Survived"] == 1) & (train_data["Sex"] == "male")] 
survived_female = train_data[(train_data["Survived"] == 1) & (train_data["Sex"] == "female")]
print( "Men survived - ", len(survived_men), "wemen survived - ", len(survived_female)) 

#Men
rich_survived_men = survived_men[survived_men["Pclass"] == 3]#47 12 with families, 2 with cabins
middle_survived_men = survived_men[survived_men["Pclass"] == 2]#17, 8 with families 4 with cabins
poor_survived_men = survived_men[survived_men["Pclass"] == 1]#45, 16 with families,39 had a cabin
rich_survived_men_with_family = rich_survived_men[rich_survived_men["SibSp"] >0]#12
middle_survived_men_with_family = middle_survived_men[middle_survived_men["SibSp"] >0]#8
poor_survived_men_with_family = poor_survived_men[poor_survived_men["SibSp"] >0]#16

#Women
rich_survived_female = survived_female[survived_female["Pclass"] == 3]#72 24 family, 4 cab
middle_survived_female = survived_female[survived_female["Pclass"] == 2]#70, 29 family, 9 cab
poor_survived_female = survived_female[survived_female["Pclass"] == 1]#91, 43  family, 78 with cabin
rich_survived_female_with_family = rich_survived_female[rich_survived_female["SibSp"] > 0]#24
middle_survived_female_with_family = middle_survived_female[middle_survived_female["SibSp"] > 0]#29
poor_survived_female_with_family = poor_survived_female[poor_survived_female["SibSp"] > 0]#43


amount of men: 577
Men survived -  109 wemen survived -  233


In [173]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]
#Best features?
features = ["Sex", "SibSp", "Cabin", "Parch", "Pclass", "Age"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
X, X_test = X.align(X_test, join='left', axis=1, fill_value=0)
model = RandomForestClassifier(n_estimators=150, max_depth=6, random_state=2025)
model.fit(X, y)
predictions = model.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
